In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

import keras_tuner


# In[42]:

# scld_window = scaler.fit_transform(window_data[0].flatten().reshape(-1,1))

def df_index_alter(df):
    df = df.sort_values(by=['Date'],ascending=True)
    # setting the index as date
    df.index = df['Date']
    # drop date column as it is now index column
    df = df.drop(['Date'],axis=1)
    return df


def data_preprocess(dataframe, cols, sequence_length,split):
    
    len_dataframe=dataframe.shape[0]
    
    i_split = int(len(dataframe) * split)
    data_train = dataframe.get(cols).values[:i_split]
    data_test  = dataframe.get(cols).values[i_split:]
    len_train  = len(data_train)
    len_test   = len(data_test)
    len_train_windows = None
    print('data_train.shape',data_train.shape)
    print('data_test.shape',data_test.shape)
    
    #get_test_data    
    data_windows = []
    for i in range(len_test - sequence_length):
        data_windows.append(data_test[i:i+sequence_length])
    data_windows = np.array(data_windows).astype(float)
     # get original y_test
    y_test_ori = data_windows[:, -1, [0]]
    print('y_test_ori.shape',y_test_ori.shape)
    
    window_data=data_windows
    win_num=window_data.shape[0]
    col_num=window_data.shape[2]
    normalised_data = []
    record_min=[]
    record_max=[]
    
    #normalize
    for win_i in range(0,win_num):
        normalised_window = []
        for col_i in range(0,col_num):
          temp_col=window_data[win_i,:,col_i]
          temp_min=min(temp_col)
          if col_i==0:
            record_min.append(temp_min)#record min
          temp_col=temp_col-temp_min
          temp_max=max(temp_col)
          if col_i==0:
            record_max.append(temp_max)#record max
          temp_col=temp_col/temp_max
          normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data=np.array(normalised_data)
    
    # normalised_data=window_data
    data_windows=normalised_data#get_test_data
    x_test = data_windows[:, :-1]
    y_test = data_windows[:, -1, [0]]
    print('x_test.shape',x_test.shape)
    print('y_test.shape',y_test.shape)
    
    #get_train_data 
    data_windows = []
    for i in range(len_train - sequence_length):
        data_windows.append(data_train[i:i+sequence_length])
    data_windows = np.array(data_windows).astype(float)
      
    window_data=data_windows
    win_num=window_data.shape[0]
    col_num=window_data.shape[2]
    
    normalised_data = []
    for win_i in range(0,win_num):
        normalised_window = []
        for col_i in range(0,col_num):
          temp_col=window_data[win_i,:,col_i]
          temp_min=min(temp_col)
          temp_col=temp_col-temp_min
          temp_max=max(temp_col)
          temp_col=temp_col/temp_max
          normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)
    normalised_data=np.array(normalised_data)
    
    # normalised_data=window_data
    data_windows=normalised_data
    x_train = data_windows[:, :-1]
    y_train = data_windows[:, -1,[0]]
    print('x_train.shape',x_train.shape)
    print('y_train.shape',y_train.shape)
    return x_train, y_train, x_test, y_test, y_test_ori, record_max, record_min


# In[43]:

    
# def call_existing_code(units, activation, dropout, lr):
#     model = keras.Sequential()
#     model.add(layers.Flatten())
#     model.add(layers.Dense(units=units, activation=activation))
#     if dropout:
#         model.add(layers.Dropout(rate=0.25))
#     model.add(layers.Dense(10, activation="softmax"))
#     model.compile(
#         optimizer=keras.optimizers.Adam(learning_rate=lr),
#         loss="categorical_crossentropy",
#         metrics=["accuracy"],
#     )
#     return model


# def build_model(hp):
#     units = hp.Int("units", min_value=32, max_value=512, step=32)
#     activation = hp.Choice("activation", ["relu", "tanh"])
#     dropout = hp.Boolean("dropout")
#     lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
#     # call existing model-building code with the hyperparameter values.
#     model = call_existing_code(
#         units=units, activation=activation, dropout=dropout, lr=lr
#     )
#     return model

def build_train_model(units, activation,lr, drop_out):
# LSTM MODEL
    input_timesteps= 20 - 1
    model = Sequential()
    # model.add(LSTM(units=units, input_shape=(input_timesteps, 1), return_sequences = True))
    model.add(LSTM(units=units, return_sequences = True))
    if drop_out:
        model.add(Dropout(rate=0.25))
    model.add(LSTM(units=units,return_sequences = True))
    model.add(LSTM(units=units,return_sequences =False))
    if drop_out:
        model.add(Dropout(rate=0.25))
    model.add(Dense(units=units, activation=activation))  
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(learning_rate=lr))
    # Fit the model
    # model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)
    
    return model
    # model.predict(x_test)
    
    
def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu","tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = build_train_model(
        units=units, activation=activation, drop_out=dropout, lr=lr
    )
    return model
    
    # In[44]:
    
    
    # model.predict(x_test)


# In[46]:

def validation(model, x_test, y_test, y_test_ori, prediction_len, record_max, record_min):

    #multi sequence predict
    data=x_test
    prediction_seqs = []
    window_size=sequence_length
    pre_win_num=int(len(data)/prediction_len)    
    for i in range(0,pre_win_num):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(0,prediction_len):
          temp=model.predict(curr_frame[newaxis,:,:])[0]
          predicted.append(temp)
          curr_frame = curr_frame[1:]
          curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
        
    # print(prediction_seqs)
    
    
    # In[48]:
    
    
    #de_predicted
    de_predicted=[]
    len_pre_win=int(len(data)/prediction_len)
    len_pre=prediction_len
    
    m=0
    for i in range(0,len_pre_win):
        for j in range(0,len_pre):
          de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
          m=m+1
    # print(de_predicted)
    
    
    # In[50]:
    
    
    error = []
    diff=y_test.shape[0]-prediction_len*pre_win_num
    
    for i in range(y_test_ori.shape[0]-diff):
        error.append(y_test_ori[i,] - de_predicted[i])
        
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val) 
        absError.append(abs(val))
    
    error_percent=[]
    for i in range(len(error)):
        val=absError[i]/y_test_ori[i,]
        val=abs(val)
        error_percent.append(val)
    
    mean_error_percent=sum(error_percent) / len(error_percent)
    accuracy=1-mean_error_percent
    
    MSE=sum(squaredError) / len(squaredError)
    RMSE = math.sqrt(MSE)
    
    return RMSE, accuracy, de_predicted
    # In[50]:

